In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
m = pd.read_csv("../../data/processed/KOBIS_개봉일람_3.csv")

In [3]:
m

,감독,전국 스크린수,전국 관객수,영화형태_단편,영화형태_옴니버스,영화형태_장편,국적_기타,국적_독일,국적_러시아,국적_미국,...,장르_어드벤처,장르_전쟁,장르_코미디,장르_판타지,등급_12세이상관람가,등급_15세이상관람가,등급_전체관람가,등급_청소년관람불가,영화구분_독립/예술영화,영화구분_일반영화
0,14.306190,1587,17613682,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1,15.172542,1978,16264944,False,False,True,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
2,15.714698,1912,14410754,False,False,True,False,False,False,False,...,False,False,False,True,True,False,False,False,False,True
3,15.908342,966,14245998,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,14.740589,2835,13934592,False,False,True,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324,7.217443,82,1362,False,False,True,True,False,False,False,...,False,False,False,False,False,True,False,False,False,True
2325,7.205635,60,1346,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2326,7.156956,71,1282,False,False,True,False,False,False,True,...,False,False,False,False,False,True,False,False,False,True
2327,7.770353,79,1254,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [4]:
# 데이터 분할
X, y = m.drop(["전국 관객수"], axis=1), m["전국 관객수"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)